# IMPORTS

In [1]:
import pandas as pd

import my_pickle as mp
import my_split as ms
import my_dataframes as md

from importlib import reload

import warnings
warnings.filterwarnings('ignore')

In [2]:
reload(md)

<module 'my_dataframes' from '/Users/gandalf/Documents/Galvanize/MatchingService/my_dataframes.py'>

# READ IN

In [5]:
# read in message data
message_df = md.get_message_df()
convo_df = md.get_conversation_df(message_df)
last_df = md.get_lastmessage_df()
user_df = md.get_user_data()

created message dataframe
created conversation dataframe with 0 known errors
created last message dataframe
created user dataframe


In [6]:
user_df.head()

,created,has_room,updated,about,activeAt,amenities,available,birthday,college,facebookId,...,I_count,I_ratio,period_count,period_ratio,question_count,question_ratio,exclaim_count,exclaim_ratio,sentence_count,sentence_ratio
uid,,,,,,,,,,,,,,,,,,,,,
00yWCOsIvK,2017-03-13 19:52:14.656,False,2018-01-12 23:08:47.553,,2017-05-31 14:36:52.153,[Laundry],NaT,1986-01-01 00:00:00,Academy of Art University,False,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
01Kc0ZYNae,2017-03-11 00:22:49.100,False,2018-01-12 23:08:51.185,,NaT,"[A/C, Dishwasher, Laundry]",NaT,1993-01-01 00:00:00,University of Arizona,True,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
01mfWG4vUm,2017-02-08 12:01:19.847,False,2018-01-12 23:08:54.367,,2017-04-07 04:00:21.540,[Laundry],NaT,1992-01-01 00:00:00,San Francisco State University,True,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
02GDyQPLII,2017-02-28 02:59:07.884,False,2018-01-12 23:08:59.864,,NaT,"[Parking, Laundry]",NaT,1997-02-18 21:00:00,University of San Francisco,True,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
04yEQfqfiB,2017-03-12 16:01:40.628,False,2018-01-12 23:09:03.164,,NaT,[Laundry],NaT,1994-01-01 00:00:00,Mission College,False,...,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN


# ECT SPLIT

In [7]:
# find the split!
cutoff = ms.ect_find_split(convo_df, .7)
print("split on:                      {}".format(pd.to_datetime(cutoff*1000000)))

# do the split!
safe_convo_df = ms.ect_make_split(convo_df, cutoff)
print("most recent convo data:        {}".format(pd.to_datetime(safe_convo_df.timestamp.max()*1000000)))

safe_message_df = ms.ect_make_split(message_df, cutoff)
print("most recent message data:      {}".format(pd.to_datetime(safe_message_df.timestamp.max()*1000000)))

# safe_lastmessage_df = ms.ect_make_split(last_df, cutoff)
# print("most recent last message data: {}".format(pd.to_datetime(safe_lastmessage_df.timestamp.max()*1000000)))

split on:                      2017-10-02 01:35:58.644000
most recent convo data:        2017-10-02 01:35:58.644000
most recent message data:      2017-10-02 01:35:58.644000


# REMOVE ROWS FROM CONVOS WITH MISSING UIDS

In [8]:
safe_convo_df = md.remove_bad_uids(safe_convo_df, user_df)  

467 rows dropped


# PICKLE EVERYTHING!

https://www.youtube.com/watch?v=yYey8ntlK_E

In [9]:
mp.json_it(user_df,'data_user')
mp.json_it(safe_message_df,'data_message')  
mp.json_it(safe_convo_df,'data_convo')  
# mp.json_it(lastmessage_df,'lastmessage_df')